In [253]:
import collections
import random

def cross(a,b):
    return [s+t for s in a for t in b]


rows = "ABCDEFGHI"
cols = "123456789"    
boxes = cross(rows,cols)
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(r,x) for r in ("ABC","DEF","GHI") for x in ("123","456","789")]
# diag units for solving diagonal sudoku
diag_units = [[chr(64 + x) + str(x) for x in range(1,10)],[chr(74 - x) + str(x) for x in range(1,10)]]
unitlist = row_units + column_units + square_units + diag_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

In [254]:
assignments = []

def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

In [255]:
def start_game(grid):
    values = game
    assert len(grid) == 81
    return dict(zip(boxes,values))

In [256]:
def grid_values(grid):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    assert len(grid) == 81
    values = []
    for item in grid:
        if item == '.':
            values.append('123456789')
        else:
            values.append(item)
    return dict(zip(boxes,values))

In [257]:
def eliminate(values):
    """
    Use Elimination constraint propagation technique to eliminate values.
    Input: The sudoku in dictionary form
    Output: Reduced sudoku puzzle in dictionary form
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for value in solved_values:
        for peer in peers[value]:
            values[peer] = values[peer].replace(values[value],"")
    return values


In [258]:
def only_choice(values):
    """
    Use Only Choice constraint propogation technique to reduce values.
    Input: The sudoku in dictionary form
    Output: Reduced sudoku puzzle in dictionary form
    """
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values
    

In [259]:
def reduce_puzzle(values):
    """
    Iteratively use eliminate and only choice strategy.
    Input: The sudoku in dictionary form
    Output: Reduced sudoku puzzle in dictionary form
    """
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])

        # Use the Eliminate Strategy
        values = eliminate(values)
        
        #  Use the Only Choice Strategy
        values = only_choice(values)
        
        #  Use the Naked Twins Strategy
        values = naked_twins(values)

        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values
    

In [260]:
def search(values):
    """
    "Using depth-first search and propagation, try all possible values..
    Input: The sudoku in dictionary form
    Output: Reduced sudoku puzzle in dictionary form
    """
    
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes): 
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt
        

In [261]:
def naked_twins(values):
    """
    "Use naked twins propagation technique to reduce puzzle
    Input: The sudoku in dictionary form
    Output: Reduced sudoku puzzle in dictionary form
    """
    # Iterate through each list of units in unitlist
    for units in unitlist:
        # Retrieve values for each unit in list
        unit_values = [values[value] for value in units]
        # Find all naked twin values
        nt = [item for item, count in collections.Counter(unit_values).items() if count > 1 and len(item)==2]
        
        # If there is not any naked twins move to the next list of units
        if not nt:
            continue
        
        #Iterate through each unit in unitlist
        for unit in units:
            for twin in nt:
                if values[unit] == twin:
                    continue
                #If unit is not a naked twin remove naked twin values
                for val in twin:
                    values[unit] = values[unit].replace(val,"")
    return values
    

In [262]:
def solve(grid):
    """
    Find the solution to a Sudoku grid.
    Args:
        grid(string): a string representing a sudoku grid.
            Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
            '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
    Returns:
        The dictionary representation of the final sudoku grid. False if no solution exists.
    """
    values = search(grid_values(grid))
    #If puzzle is solved return the values in dictionary form, else return False
    if all(len(values[s]) == 1 for s in boxes):
        return values
    else:
        return False

In [263]:
if __name__ == '__main__':
    diag_sudoku_grid = game
    display(solve(diag_sudoku_grid))

    try:
        from visualize import visualize_assignments
        visualize_assignments(assignments)

    except SystemExit:
        pass
    except:
        print('We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.')

3 6 5 |8 4 2 |9 7 1 
8 2 7 |9 1 5 |3 4 6 
4 9 1 |7 6 3 |2 5 8 
------+------+------
5 7 6 |4 9 8 |1 3 2 
2 8 4 |3 5 1 |6 9 7 
9 1 3 |6 2 7 |4 8 5 
------+------+------
6 5 9 |2 7 4 |8 1 3 
1 3 2 |5 8 9 |7 6 4 
7 4 8 |1 3 6 |5 2 9 
We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.


In [264]:
game ='267945381853716249491823576576438192384192657129657438642379815935281764718564923'
point = ['.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.','.',]
boxes_solution = dict(zip(boxes,game))
boxes_new_game = dict(zip(boxes,point))

def random_initialn(qty_number):    
    initial_n = []
    while len(initial_n) != qty_number:
        r = random.randint(0, 80)
        if r not in initial_n:
            initial_n.append(r)
    return initial_n

def new_game(random_list):
    
    for n in random_list:
        boxes_new_game[boxes[n]] = boxes_solution[boxes[n]]
    return boxes_new_game

def improve_list(values):  
    boxes_l = ''
    for box in boxes:
        boxes_l = boxes_l + str(boxes_new_game[box])
        #print(boxes_l)
    all_games.append(boxes_l) 
    return all_games

In [265]:
nivel = 15
N_games = 10
n_games = 0
initial_n = []
all_games = []
value_str = []
game = ''

while n_games < N_games:
    values = random_initialn(nivel)
    values = new_game(values)
    values = improve_list(values)
    n_games += 1
    boxes_new_game = dict(zip(boxes,point))
    
print(all_games)

['...........37...4........7.............1.............8..........352....47185.4...', '2..94.3........2......2...65.........8..9.........7....4..........2......1.....2.', '.............1......1..3...57....1........6..1.96........37.............7...6...3', '.......8.....1.2......23......4...92....9....1..........2....1.9..............9.3', '2.....3....37.....4...2......6....9......2.......574...4...9............7........', '..7...........624.4.1.........4.8.......9..5................8..93..8...4.........', '.6.............2..4.....5....6........4192...........8.....98....5.........5....3', '2.......18..7..............5....8....84.9.6.....6..............9......6..1....9..', '..79..............491.2.5....6.............5.........86...7...........6....5...2.', '....4.381.......4...1.......................7.29....38..23...............1..6....']


In [266]:
#Teste dos jogos criados
n = 0
while n < len(all_games):
    game = all_games[n]
    print (display(start_game(game)))
    n += 1

. . . |. . . |. . . 
. . 3 |7 . . |. 4 . 
. . . |. . . |. 7 . 
------+------+------
. . . |. . . |. . . 
. . . |1 . . |. . . 
. . . |. . . |. . 8 
------+------+------
. . . |. . . |. . . 
. 3 5 |2 . . |. . 4 
7 1 8 |5 . 4 |. . . 
None
2 . . |9 4 . |3 . . 
. . . |. . . |2 . . 
. . . |. 2 . |. . 6 
------+------+------
5 . . |. . . |. . . 
. 8 . |. 9 . |. . . 
. . . |. . 7 |. . . 
------+------+------
. 4 . |. . . |. . . 
. . . |2 . . |. . . 
. 1 . |. . . |. 2 . 
None
. . . |. . . |. . . 
. . . |. 1 . |. . . 
. . 1 |. . 3 |. . . 
------+------+------
5 7 . |. . . |1 . . 
. . . |. . . |6 . . 
1 . 9 |6 . . |. . . 
------+------+------
. . . |3 7 . |. . . 
. . . |. . . |. . . 
7 . . |. 6 . |. . 3 
None
. . . |. . . |. 8 . 
. . . |. 1 . |2 . . 
. . . |. 2 3 |. . . 
------+------+------
. . . |4 . . |. 9 2 
. . . |. 9 . |. . . 
1 . . |. . . |. . . 
------+------+------
. . 2 |. . . |. 1 . 
9 . . |. . . |. . . 
. . . |. . . |9 . 3 
None
2 . . |. . . |3 . . 
. . 3 |7 . . |. . . 
4 . . |. 2 . |

In [267]:
#Teste dos jogos criados
n = 0
while n < len(all_games):
    if __name__ == '__main__':
        diag_sudoku_grid = all_games[n]
        display(solve(diag_sudoku_grid))

        try:
            from visualize import visualize_assignments
            visualize_assignments(assignments)

        except SystemExit:
            pass
        except:
            print('We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.{}'.format(n))
    n += 1

2 8 7 |4 5 9 |3 6 1 
9 6 3 |7 1 2 |8 4 5 
1 5 4 |8 3 6 |9 7 2 
------+------+------
8 2 1 |9 4 5 |6 3 7 
5 7 6 |1 8 3 |4 2 9 
3 4 9 |6 2 7 |1 5 8 
------+------+------
4 9 2 |3 7 1 |5 8 6 
6 3 5 |2 9 8 |7 1 4 
7 1 8 |5 6 4 |2 9 3 
We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.0
2 5 1 |9 4 6 |3 7 8 
6 3 9 |8 7 5 |2 1 4 
8 7 4 |3 2 1 |5 9 6 
------+------+------
5 2 6 |1 8 3 |7 4 9 
4 8 7 |5 9 2 |1 6 3 
1 9 3 |4 6 7 |8 5 2 
------+------+------
9 4 2 |7 5 8 |6 3 1 
3 6 5 |2 1 4 |9 8 7 
7 1 8 |6 3 9 |4 2 5 
We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.1
9 6 2 |7 4 8 |5 3 1 
3 5 7 |9 1 6 |8 2 4 
8 4 1 |5 2 3 |9 7 6 
------+------+------
5 7 6 |2 3 4 |1 9 8 
4 2 3 |1 8 9 |6 5 7 
1 8 9 |6 5 7 |3 4 2 
------+------+------
6 1 5 |3 7 2 |4 8 9 
2 3 4 |8 9 1 |7 6 5 
7 9 8 |4 6 5 |2 1 3 
We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.2
2 3 5 |9 6 4 